In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd

from tqdm import tqdm
import re
tqdm.pandas()

In [51]:
harmful_cat_dict = \
{'S1': 'Violent Crimes',
 'S2': 'Non-Violent Crimes',
 'S3': 'Sex-Related Crimes',
 'S4': 'Child Sexual Exploitation',
 'S5': 'Defamation',
 'S6': 'Specialized Advice',
 'S7': 'Privacy',
 'S8': 'Intellectual Property',
 'S9': 'Indiscriminate Weapons',
 'S10': 'Hate',
 'S11': 'Suicide & Self-Harm',
 'S12': 'Sexual Content',
 'S13': 'Elections',
 'S14': 'Code Interpreter Abuse'
}

In [96]:
model_id = "meta-llama/Llama-Guard-3-8B-INT8"
device = "cuda"
dtype = torch.bfloat16
CHUNK_TOKENS = 1024
PADDING = 16

In [97]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, device_map=device, quantization_config=quantization_config)

2025-01-18 13:31:23.029910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737196283.065128 3488718 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737196283.078786 3488718 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 13:31:23.105431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.qu

In [105]:
def split_text_into_chunks(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = [tokens[i:i + CHUNK_TOKENS + PADDING] for i in range(0, len(tokens), CHUNK_TOKENS)]
    text_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    
    return text_chunks

In [115]:
def moderate(content):
    
    chunks = split_text_into_chunks(content)
    result = {'harmful': False, 'harmfull_categories': set()}
    
    for chunk in chunks:
        
        chat = [{"role": "user", "content": content}]
    
        input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)[:,:1024]
        output = model.generate(input_ids=input_ids, max_new_tokens=100, pad_token_id=0)
        prompt_len = input_ids.shape[-1]
        result_str = tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)
        
        if 'unsafe' in result_str:
            result['harmful'] = True
            pattern = r'\S\d+'
            result['harmfull_categories'] = result['harmfull_categories']\
            .union({harmful_cat_dict[el] for el in re.findall(pattern, result_str)})

    result['harmfull_categories'] = list(result['harmfull_categories'])
        
    return result

In [116]:
moderate('как сделать хакерскую атаку?')

{'harmful': True, 'harmfull_categories': ['Non-Violent Crimes']}

In [117]:
df = pd.read_csv('data/processed_resume.csv')

In [119]:
df['llama_guard3_7b_q_resp'] = df['text_data'].progress_apply(lambda el: moderate(el))

100%|██████████| 258/258 [35:06<00:00,  8.16s/it] 


In [122]:
df['llama_guard_harmful'] = df['llama_guard3_7b_q_resp'].apply(lambda el: el['harmful'])

In [124]:
df['llama_guard_harmful_cats'] = df['llama_guard3_7b_q_resp'].apply(lambda el: el['harmfull_categories'])

In [127]:
df['llama_guard_harmful'].value_counts()

llama_guard_harmful
False    246
True      12
Name: count, dtype: int64

In [130]:
df[df['llama_guard_harmful']].text_data.tolist()

['Опыт работы: май 2018 – по настоящее время 6 лет 7 месяцев Начальник отдела резерва Банк Открытие Обязанности: Организация и управление деятельностью отдела/руководителей групп в составе отдела (группа кассиров/группа менеджеров ФЛ/группа мобильных контролеров). Руководитель оперативной группы по проверкам Филиала Северо-Западный. Выездные комплексные проверки ДО/РОО по направлениям: операционно-кассовая работа / банковские карты / ИБС / иное. Руководитель группы координации и поддержки сети по качеству и клиентскому сервису. Сопровождение выполнения задач дополнительными офисами Филиала: CSI / Операционная эффективность / Продуктивность МРБ / Аудит стандартов содержания офисов / Фотомониторинг / Жалобы и претензии клиентов / Речевая аналитика / Selvery. Подготовка материалов и проведение комитетов на уровне руководства Филиала - комитет по операционным рискам / комитет по качеству клиентского обслуживания. сентябрь 2015 – май 2018 2 года 8 месяцев Начальник отдела резерва Банк ВТБ24

In [134]:
df['llama_guard_harmful_cats'].value_counts()

llama_guard_harmful_cats
[]                             248
[Privacy]                        8
[Child Sexual Exploitation]      1
[Specialized Advice]             1
Name: count, dtype: int64

In [146]:
df[df['llama_guard_harmful_cats'].apply(lambda el: len(el) > 0 and el[0] == 'Child Sexual Exploitation')]\
.text_data.tolist()

['Опыт работы: ноябрь 2024 – по настоящее время 1 месяц Инструктор по спорту ГБУ ДО ЦОО "Березняки" Обязанности: Работа иструктором по спорту с детьми 8-17 лет, в период проведения осенней каникулярной смены с 26 октября по 03 ноября 2024 года. В период проведения дополнительной осенней смены с 11.10.224 по 24.10.2024 для детей г. Енакиево (ДНР), находящихся в трудной жизненной ситуации. В период проведения зимней каникулярной смены с 01 января по 08 января 2025 года.\xa0 Достижения: Благодарственое письмо от Директора ГБУ ДО ЦОО "Березняки" Федоровой Натальи Владимировны, ректору ЛГУ им. А.С.Пушкина Г.В. Двасу.\nДолжность: Тренер - преподаватель по легкой атлетике\nОбразование: 2025 «Ленинградский государственный университет имени А.С. Пушкина» Факультет:\xa0 Физической культуры и спорта Специальность:\xa0 Тренер - преподаватель Квалификация:\xa0 Магистр Дипломная работа:\xa0 Изучение особенностей техники бега на 800 метров у легкоатлетов первого разряда\nДПО:   Не указано']

In [147]:
df[df['llama_guard_harmful_cats'].apply(lambda el: len(el) > 0 and el[0] == 'Specialized Advice')]\
.text_data.tolist()

['Опыт работы: ноябрь 2020 – декабрь 2021 1 год 1 месяц терапевт инфекционного отделения стационара ГБУЗ МО Домодедовская ЦГБ Обязанности: консультативная работа (терапевтическая патология коморбидных пациентов отделения). Работа преимущественно с эндокринной и кардиальной нозологией.\xa0 октябрь 2019 – сентябрь 2020 11 месяцев Врач неотложной помощи в терапевтическом отделении Ооо медицинские услуги, клиника Baltmed Обязанности: Круглосуточная курация пациентов в терапевтическом отделении многопрофильного ЛПУ. Преимущественно, неотложные кардиологические состояния.\xa0 октябрь 2019 – август 2020 10 месяцев врач-терапевт по оказанию неотложной помощи ГБУЗ Клиническая инфекционная больница имени Боткина Обязанности: Осуществление плановой и неотложной круглосуточной терапевтической помощи пациентам стационара. январь 2019 – июль 2019 6 месяцев врач-терапевт выездной терапевтической службы ООО ЦСМ "Медика" Обязанности: Лечебно-консультативная помощь пациентам на дому ноябрь 2016 – май 20

In [151]:
#### Это всё ложные срабатывания